In [ ]:
import numpy as np
import holoviews as hv; hv.extension('bokeh', logo=False)
import panel as pn;     pn.extension()
from panel.interact import interact

import sympy as sp
from IPython.display import display, Latex, Math

from julia.api import Julia
jl = Julia(compiled_modules=False)
from julia import Main

def format_matrix_with_parentheses(A):
    A_latex = sp.latex(A)
    return A_latex.replace("\\begin{bmatrix}", "\\begin{pmatrix}").replace("\\end{bmatrix}", "\\end{pmatrix}")

%load_ext julia.magic

In [2]:
%%julia
using Pkg, Revise
gla_dir = "../GenLinAlgProblems"
Pkg.activate(gla_dir)
using GenLinAlgProblems, LinearAlgebra, LaTeXStrings, Latexify, Markdown

  Activating project at `~/elementary-linear-algebra/GenLinAlgProblems`



<div style="float:center;width:100%;text-align:center;">
<strong style="height:100px;color:darkred;font-size:40px;">Schur Decomposition: A Gateway to Eigenvalues and Stability</strong>
</div>
    

# 1. Introduction

## 1.1 Schur's Lemma

The eigendecomposition of a square matrix $A$ of size $N \times N$ exists only if $A$ has a complete set<br>
of $N$ linearly independent eigenvectors.

One approach to address this even when the matrix is degenerate<br> (and actually the most useful in numerical analysis) is using a Schur decomposition:<br>
Instead of a **diagonal form**, this decomposition obtains an **upper triangular form** of the matrix $A$.<br>
Interestingly, this can be achieved with a set of orthonormal basis vectors.

<div style="float:left;width:100%;background-color:#F2F5A9;color:black;">

**Schur's Lemma:** Any square matrix $A$ has the form $\;\; A = Q T Q^H,\;\;$
where
- $Q$ is unitary, i.e.,  $Q^H Q = I$
- $T$ is upper triangular
</div>

**Examples:**
* $A=\begin{pmatrix} 4 & \;\;1 \\ 2 & \;\;3\end{pmatrix}, \quad
Q = \frac{1}{\sqrt{2}} \left(\begin{array}{rr} 1 & -1 \\ 1 & 1 \end{array}\right), \quad T = \begin{pmatrix}5 & 1 \\ 0 & 2\end{pmatrix}\;\;$ satisfies $A = Q\ T\ Q^t$
* $A=\left(\begin{array}{rr} 1 & -3 \\ 2 & 4\end{array}\right), \quad
Q = \frac{1}{\sqrt{2}} \left(\begin{array}{rr}  i & -i \\ 1 & 1  \end{array}\right), \quad T = \begin{pmatrix} 1 + 3i & 0 \\ 0 & 1 - 3i\end{pmatrix}\;\;$ satisfies $A = Q\ T\ Q^t$

## 1.2 A Constructive Proof

The key idea of Schur's Lemma is to **iteratively construct the triangular matrix** $T$<br> using
orthogonal matrices $𝑄_i$ while maintaining the similarity transform $A = Q\ T\ Q^t$

### 1.2.1 Use an Eigenvector to Introduce Zeros in the First Column

**Reminder:** Any $N \times N$ matrix $A$ has at least one eigenpair $(\lambda,x)$ for every distinct eigenvalue $\lambda$.<br><br>
Let us chose one such eigenpair, and without loss of generality assume that $x$ has unit length, i.e., $x^t x = 1$.<br><br>
Extend $\left\{ x \right\}$ to a full orthonormal basis of $N$ vectors. Note that since $A$ may have complex eigenvalues,<br>
$\qquad$ the resulting matrix
$Q = \begin{pmatrix} x &q_2&q_3&\dots &q_N\end{pmatrix} = \begin{pmatrix} x & \tilde{Q}\end{pmatrix}$
may have complex entries.

$\qquad \begin{aligned}
Q^H A Q &= \begin{pmatrix} x^H \\ \tilde{Q}^H \end{pmatrix}\  A\ \begin{pmatrix} x & &\tilde{Q}\end{pmatrix}\qquad & \\
&=         \begin{pmatrix} x^H \\ \tilde{Q}^H \end{pmatrix} \begin{pmatrix} A x & A \tilde{Q} \end{pmatrix} & \text{ now use } A x = \lambda x \;\; \text{ and } x^t x = 1 & \\
&=         \begin{pmatrix} \lambda & x^H A \tilde{Q}^H \\
                           \lambda \tilde{Q}^H x & \tilde{Q}^H A \tilde{Q}\end{pmatrix} &
                           \text{ but } x \perp q_2, q_3, \dots q_n \\ 
&=          \begin{pmatrix} \lambda & x^H A \tilde{Q}^H \\
                           0 & \tilde{Q}^H A \tilde{Q}\end{pmatrix} &
\end{aligned}$

Observe that $\tilde{a}^H = x^H A \tilde{Q}^H$ is a row vector, and $\;\;\tilde{A} = \tilde{Q}^H A \tilde{Q}\;\;$ is a matrix of size $(N-1) \times (N-1)$, i.e.,

$\qquad Q^H A Q = \begin{pmatrix} \lambda & \tilde{a}^H \\ 0 & \tilde{A} \end{pmatrix}$.

### 1.2.2 Repeat with $\tilde{A}$

To see that we can continue this process with successively smaller matrices $\tilde{A}$:
* Let $Q_1^H A Q_1 = \begin{pmatrix} \lambda_1 & \tilde{a_1}^H \\ 0 & \tilde{A}_1 \end{pmatrix}$
* obtain $\;\;\tilde{Q}_2^H \tilde{A}_1 \tilde{Q}_2 = \begin{pmatrix} \lambda_2 & \tilde{a}_2^H \\ 0 & \tilde{A}_2 \end{pmatrix}$ for some given eigenpair $(\lambda_2, x_2)$ of $\tilde{A}_1$
* Set $Q_2 = \begin{pmatrix} 1 & 0 \\ 0 & \tilde{Q_2}\end{pmatrix}\;\;$ and therefore
$\;\;(Q_2 Q_1)^H A (Q_1 Q_2) = \begin{pmatrix} \lambda_1 & \dots & \dots \\
                0          & \lambda_2 & \dots \\
                0          & 0 & \tilde{A}_3 \end{pmatrix}$
* repeat this process for each matrix $\tilde{A}_i,\;\;$ resulting in an upper triangular matrix<br>
  $\;\;T = (Q_N \dots Q_2 Q_1)^H A (Q_1 Q_2) =
  \begin{pmatrix} \lambda_1 & \dots     & \dots  &  \dots  &  \dots \\
                 0          & \lambda_2 & \dots  &  \dots  &  \dots \\
                 0          & 0         & \ddots &  \dots  &  \dots \\
                 0          & 0         &  0     & \ddots  &  \dots \\
                 0          & 0         &  0     & \dots & \lambda_N
  \end{pmatrix}$

In [ ]:
%%julia
"""
given a vector v, obtain a unitary matrix Q with v̂ as its first column
"""
function unitary_matrix_from_vector(v::AbstractVector)
    n       = length(v)
    A       = [ v/norm(v) one(eltype(v))I(n)]
    Q, R    = qr(A)
    return Q[:, 1:n]
end

In [ ]:
%%julia
"""
given a matrix A, obtain a Schur triangularization
"""
function schur_triangularization(A::AbstractMatrix)
    # Ensure A is square
    n, m = size(A)
    @assert n == m "Matrix A must be square"

    # Initialize Q as the identity matrix
    Q = one(eltype(A)) * I(n)

    # Start iterating over submatrices
    for i in 1:n-1
        # Extract the current submatrix
        subA = A[i:end, i:end]
        
        # Compute the eigenvector of the first eigenvalue
        λ, V = eigen(subA)
        v = V[:, 1]  # First eigenvector

        # Construct a unitary matrix using v
        U_sub = unitary_matrix_from_vector(v)  # Unitary for the submatrix
        U     = Matrix( one(eltype(U_sub)) * I(n))  # Full-size identity matrix
        U[i:end, i:end] = U_sub  # Embed the submatrix into U
        py_show( L"\text{Step }", i, L":\quad \tilde{A} =", subA, L", \quad v_%$i =", round.(v,digits=2),  L", \quad Q_%$i =", round.(U,digits=2), inline=true)

        # Apply the unitary transformation
        A = U' * A * U
        Q = Q * U
    end

    # T is the upper triangular matrix after transformations
    T = A
    return Q, T
end

A   = [1 3 0 ; -3 1 0; -2 4 0 ]
py_show(L"\text{Triangularize } A =", A, color="blue", inline=true)
Q, T = schur_triangularization(A)
py_show( L"A = Q T Q^H, \quad Q =", round.(Q,digits=3), L", \quad T = ", round.(T, digits=3), color="blue", inline=true)
@show A ≈ Q*T*Q';

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

A ≈ Q * T * Q' = true

# 3. Examples

### Example 1: Simple Matrix
Let $A = \begin{bmatrix} 4 & 1 \\ 2 & 3 \end{bmatrix}$.
The Schur decomposition yields:

$Q = \begin{bmatrix} ... \end{bmatrix}$

$T = \begin{bmatrix} ... \end{bmatrix}$

Try it interactively above.
    

# 4. Take Away

### Summary
- The Schur decomposition expresses a square matrix $A$ as $A = Q T Q^H$.
- The diagonal of $T$ contains the eigenvalues of $A$.
- $Q$ is unitary, preserving numerical stability.

### Why Use Schur Decomposition?

While eigendecomposition also expresses a matrix in terms of its eigenvalues and eigenvectors, it has some limitations:
1. **Existence**:
   - Eigendecomposition exists only for diagonalizable matrices. Non-diagonalizable matrices, such as those with defective eigenvalues, cannot be decomposed using eigendecomposition.
   - Schur decomposition, however, exists for every square matrix, regardless of diagonalizability.

2. **Stability**:
   - Schur decomposition involves unitary matrices, which are numerically stable because they preserve lengths and angles during computations. This is especially important in applications requiring high precision.

3. **Practicality**:
   - The Schur decomposition retains more structure in the triangular matrix \( T \) compared to eigendecomposition, where the matrix is diagonal. This makes it a versatile tool in numerical linear algebra, particularly in iterative algorithms like QR or spectral analysis.

### Applications of Schur Decomposition

The Schur decomposition is fundamental in various areas, including:
- **Eigenvalue Computation**: Eigenvalues of $A$ are readily found as the diagonal elements of $T$.
- **Control Theory**: Stability of a system can be analyzed by examining the eigenvalues of $A$ through its Schur form.
- **Spectral Analysis**: Provides a basis for understanding the spectrum of a matrix.
- **Matrix Functions**: Facilitates the computation of matrix exponentials and logarithms.